In [1]:
import sys
import os
import astropy.io.fits as fits
import math
import os.path
from astropy.visualization import astropy_mpl_style
from astropy.utils.data import get_pkg_data_filename
from astropy.io import fits
# import matplotlib as plt
import os.path
import glob
import matplotlib.pyplot as plt
import operator
from matplotlib.colors import LogNorm
from photutils.datasets import make_4gaussians_image
from photutils import centroid_com, centroid_1dg, centroid_2dg
import numpy as np
from astropy.visualization import PercentileInterval

In [2]:
# Opening the file

# test with one bcd file
dir_bcd ='/Users/melaniapena/Rsrch/Luhman_16_Research/only_bcd_files/SPITZER_I1_50037760_0000_0000_2_bcd.fits'
fits_file = fits.open(dir_bcd)

## displays HDUL types within the fits file.
fits_data = get_pkg_data_filename(dir_bcd)
fits.info(fits_data)

list_Primary = fits_file[0]
# list_Primary.header


image_data= fits.getdata(fits_data, ext=0)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/melaniapena/Rsrch/Luhman_16_Research/only_bcd_files/SPITZER_I1_50037760_0000_0000_2_bcd.fits'

In [ ]:
# list_Primary.header

In [ ]:
## pulls out specific information when needed.
frame_t = fits_file[0].header['FRAMTIME']
print('Frame Time:',frame_t)

bmjd_obs = fits_file[0].header['BMJD_OBS']
print('bmjd observation:', bmjd_obs)


flux_conv= fits_file[0].header['FLUXCONV']
print('Flux Conversion',flux_conv)

gain = fits_file[0].header['GAIN']
print('Gain',gain)

exp_time= fits_file[0].header['EXPTIME']
print('Exp-time', exp_time)

In [ ]:
from collections import deque
i = 0
j_final = 10

frameday = frame_t/(3600*24)
shift_bmjd = bmjd_obs + 0.5* frameday
list_bmjd =[shift_bmjd]

print(list_bmjd[0])

In [ ]:
# Adding 
for j in range(j_final):
    value = list_bmjd[0] + (j+1)*frameday
    bmjd_arr = list_bmjd.append(value)
print(list_bmjd)

In [ ]:
# flux conversion

conversion_factor = gain*exp_time/flux_conv
print(conversion_factor)
counts_image = image_data * conversion_factor

In [ ]:
print(counts_image)
add_cnt_im = counts_image + 200
scale = 100
interval = PercentileInterval(scale)
interval.get_limits(counts_image[0,:,:])
sclarr = interval(counts_image[0,:,:])
plt.imshow(sclarr)

# plt.show()

# # print(counts_image[0,:,:].shape)
# # we take the centroid in this portion..

In [ ]:
# definitions
def slicing_im(npix,box):
    npix = npix #total pixels
    box = box #changes the display of arrays
    result= (npix - box)/2
#     print(result)
#     print(result)
    if result % 1 == 0:
        end_num = int(result)
        strt_num = end_num
        new_st = strt_num
        new_end = npix - end_num
    else:
#         end_num = int(round(result,0))
        end_num = result +0.5
        strt_num = end_num - 1
        new_st = strt_num 
        new_end = npix - end_num
#     print("total pixels is: ",npix,'\nLeft side start by:',new_st,"\nRight side end by:",new_end)
    strt = int(new_st)
    end = int(new_end)
    return strt, end

def scale_image(scale,time, data_image):
    scale_num = scale
    interval = PercentileInterval(scale_num)
    interval.get_limits(data_image[time,:,:])
    scale_array = interval(data_image[time,:,:])
    return scale_array 

In [ ]:
# npix = 32
# box = 3
# # slicing
# strt, end = slicing_im(npix,box)
# print(strt,end)
# im2 = counts_image[0,strt:end,strt:end] 
# plt.imshow(im2)

In [ ]:
# npix = 32
# box =20
# strt, end = slicing_im(npix,box)
# print(strt, end)
# im1 = counts_image[0,:,:] #time,x,y
# im2 = counts_image[0,strt:end,strt:end] # slice to smaller box
# plt.subplot(2, 1, 1)
# plt.imshow(im2)
# plt.subplot(2, 1, 2)
# plt.imshow(sclarr)

# print(im2)

In [ ]:
# radial profile fit
def radial_profile(data,x1,y1):
    y, x = np.indices((data.shape))
    r = np.sqrt((x - x1)**2 + (y - y1)**2)
    r = r.astype(np.int)
    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel())
    radialprofile = tbin / nr
    return radialprofile 

In [ ]:
from photutils import fit_2dgaussian

# from scipy import optimize
npix = 32
box = 9
# slicing
strt, end = slicing_im(npix,box)
# print(strt,end)
im2 = counts_image[0,strt:end,strt:end] 
# plt.imshow(im2)

# ----------------------------------
# percentile
# scaled_im = scale_image(98,0,counts_image)
# data = scaled_im
# plt.imshow(scaled_im)

time =0
scale_num = 100
interval = PercentileInterval(scale_num)
interval.get_limits(counts_image[time,strt:end,strt:end])
scale_array = interval(counts_image[time,strt:end,strt:end])
# plt.imshow(scale_array)


# data = counts_image[0,strt:end,strt:end]
# x1, y1 = centroid_com(data)
# x3, y3 = centroid_2dg(data)
# print('COM: ',x1,y1)
# print('2d Gaussian: ',x3,y3)
# radial_prof = radial_profile(data,x1,y1)
# radial_prof_2 = radial_profile(data,x3,y3)
# guass_params = fit_2dgaussian(counts_image[0,:,:])
# guass_params

In [ ]:
time_i = 0
scale_num = 100
COM_arr_x=[]
npix = 32
start_i = 3
steps_i = 2

centroid_com_arr_x1 = []
centroid_com_arr_y1 = []
centroid_2dG_arr_x2 = []
centroid_2dG_arr_y2 = []

for i_box in range(start_i,npix,steps_i):
    strt, end = slicing_im(npix,i_box)
    interval = PercentileInterval(scale_num) ###
    interval.get_limits(counts_image[time_i,strt:end,strt:end])
    scale_array = interval(counts_image[time_i,strt:end,strt:end])
#     sliced_image = counts_image[0,strt:end,strt:end] 
    result = scale_array
    if i_box < 3:
        print('too small of an imgae to calculate\n')
    elif i_box >= 3:
        x1, y1 = centroid_com(result)
        x2, y2 = centroid_2dg(result)
        print(i_box,strt,end)
        print('Box Size: ', i_box,'x',i_box)
        print('COM: ',x1,y1)
        print('2dG: ',x2,y2,'\n')
#         ave_com = (x1+y1)/2
#         ave_2dG = (x2+y2)/2
        centroid_com_arr_x1.append(x1)
        centroid_com_arr_y1.append(y1)
        centroid_2dG_arr_x2.append(x2)
        centroid_2dG_arr_y2.append(y2)
        plt.imshow(result)
        plt.show()  

In [ ]:
# num = 0
# centroid_com_arr_x = []
# centroid_com_arr_y = []
# time_i = 0
# scale_num = 100
# npix = 32
# start_i = 3
# steps_i = 2

# # for time_i in range(0,64):
# #     for i_box in range(start_i,npix,steps_i):
# #         strt, end = slicing_im(npix,i_box)
# #         interval = PercentileInterval(scale_num) ###
# #         interval.get_limits(counts_image[time_i,strt:end,strt:end])
# #         scale_array = interval(counts_image[time_i,strt:end,strt:end])
# #         sliced_image = counts_image[0,strt:end,strt:end] 
# #         x1, y1 = centroid_com(result)
# #         x2, y2 = centroid_2dg(result)
# #         print(i_box,strt,end)
# #         print('Box Size: ', i_box,'x',i_box)
# #         print('COM: ',x1,y1)
# #         print('2dG: ',x2,y2,)
# #         print(time_i)
# #         centroid_com_arr_x.append(x1)
# # #         centroid_com_arr_y.append(y1)
# #         num= num+1
# #         print('num: ',num,'\n')

# for time_i in range(0,64):        
#     for i_box in range(start_i,npix,steps_i):
#         strt, end = slicing_im(npix,i_box)
#         interval = PercentileInterval(scale_num) ###
#         interval.get_limits(counts_image[time_i,strt:end,strt:end])
#         scale_array = interval(counts_image[time_i,strt:end,strt:end])
#         sliced_image = counts_image[0,strt:end,strt:end] 
#         result = scale_array
#         if i_box < 3:
#             print('too small of an image to calculate\n')
#         elif i_box >= 3:
#             x1, y1 = centroid_com(result)
#             x2, y2 = centroid_2dg(result)
#             print(i_box,strt,end)
#             print('Box Size: ', i_box,'x',i_box)
#             print('COM: ',x1,y1)
#             print('2dG: ',x2,y2)
#             centroid_com_arr_x.append(x1)
# #             centroid_com_arr_y.append(y1)
# #             centroid_2dG_arr_x.append(x2)
# #             centroid_2dG_arr_y.append(y2)
#             num = num+1
#             print(num,time_i,'\n')
# #             plt.imshow(result)
# #             plt.show()

In [ ]:
# # ------works-----
# print(len(centroid_com_arr_x))
# # print(centroid_com_arr_x)
# cen_test = centroid_com_arr_x[0:15]
# # print(cen_test)
# start_ind =0 
# end_ind = 15
# name_of_var = "cen_arr_x"
# empty_dict = {}
# for i in range(0,64):
#     empty_dict[name_of_var+'_'+str(i)] = centroid_com_arr_x[start_ind:end_ind]
#     start_ind = start_ind + 15
#     end_ind = end_ind+15
# print(empty_dict['cen_arr_x_63'])
# # empty_dict
# # # ---------

In [ ]:
def diff_arr(array):
    new_array = []
    for i in range(len(array)):
        result = array[i]-i-1
        new_array.append(result)
    return new_array

In [ ]:
cen_com_x = diff_arr(centroid_com_arr_x1)
cen_com_y = diff_arr(centroid_com_arr_y1)
cen_2dG_x = diff_arr(centroid_2dG_arr_x2)
cen_2dG_y = diff_arr(centroid_2dG_arr_y2)

box_size_arr = np.linspace(3,31,15)
# difference plot
# plt.plot(box_size,cen_com_x,'.')
# plt.plot(box_size,cen_com_y,'.')
# plt.plot(box_size,cen_2dG_x,'.')
# plt.plot(box_size,cen_2dG_y,'.')

# will determine what box size we should use
print('at time slice: ',time_i)
fig = plt.figure(figsize=(10,5))
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)
ax1.plot(box_size_arr,cen_com_x,'.',label = 'COM: x')
ax1.plot(box_size_arr,cen_com_y,'.',label = 'COM: y')
ax2.plot(box_size_arr,cen_2dG_x,'.',label = '2dG: x')
ax2.plot(box_size_arr,cen_2dG_y,'.',label = '2dG: y')
ax1.set_title('COM')
ax2.set_title('2dG')
plt.show
# best used at box size 9 or 10

In [ ]:
from photutils import CircularAperture
from photutils import aperture_photometry

# print(box_size_arr,'\n')
# print(centroid_2dG_arr_x2,'\n')
# print(centroid_2dG_arr_y2,'\n')
# ------------sample below
radius = 13
val_2dg_x = centroid_2dG_arr_x2[14]
val_2dg_y = centroid_2dG_arr_y2[14]

# for positions we will take the 2dg Gaussian outputs.
positions = [(val_2dg_x, val_2dg_y)]
apertures = CircularAperture(positions, r=radius)
# print(apertures)
phot_table = aperture_photometry(counts_image[0,:,:],apertures)

print(phot_table)

# ---- 
apert_sum_r13 = 385848.1178699848 #radius: 13
apert_sum_r20 = 311819.7046191514 #radius: 20
apert_diff = -apert_sum_r13 + apert_sum_r20
# print('\n',apert_diff)
# ---

# radius_arr = np.linspace(1,15,15)
# radius_arr
# counts_image[0,:,:].shape

# print(phot_table['aperture_sum'])

# ------------

In [ ]:
#  we will create a definition that will return a value

def aperture_sum(radius, data, val_x, val_y):
    R = radius
    val_2dg_x = val_x
    val_2dg_y = val_y
    # for positions we will take the 2dg Gaussian outputs.
    positions = [(val_2dg_x, val_2dg_y)]
    apertures = CircularAperture(positions, r=R)
    # print(apertures)
    phot_table = aperture_photometry(data,apertures)
    return phot_table['aperture_sum']
    

In [ ]:
# radigan was verfiying it 
# plt.imshow(counts_image[0,:,:])
# im=counts_image[0,:,:]
# xcen = centroid_2dG_arr_x2[14]
# ycen = centroid_2dG_arr_y2[14]
# pos = [(xcen, ycen)]
# rad = 11.
# ap = CircularAperture(pos, r=rad)
# print(aperture_photometry(im,ap))

In [ ]:
r_1 = 5
r_2 = 22
r_3 = 23
t = 20
aperture_1 = aperture_sum(r_1,counts_image[t,:,:], centroid_2dG_arr_x2[14], centroid_2dG_arr_y2[14]) #total counts
aperture_2 = aperture_sum(r_2,counts_image[t,:,:], centroid_2dG_arr_x2[14], centroid_2dG_arr_y2[14])
aperture_3 = aperture_sum(r_3,counts_image[t,:,:], centroid_2dG_arr_x2[14], centroid_2dG_arr_y2[14])


#Area of aperture,
area_1 = math.pi * r_1**2
# Background in annulus from r_2 to r_3
background_counts = aperture_3 - aperture_2
# Area of Annulus
area_2 = math.pi*(r_3**2 - r_2**2) #also known as npix
# subtracting background: 
backgrnd_subtract = aperture_1 - background_counts*area_1/area_2


print('radius 1:',aperture_1,'\nradius 2: ',aperture_2,'\nRadius 3:',aperture_3,'\n\n\n')
print('area one:',area_1,'\n')
print('background counts:\n',background_counts,'\n')
print('area two:',area_2,'\n')
print('background subtraction:\n',backgrnd_subtract)

In [ ]:
# background divided by area 



# background per counts
b = (aperturse_3 - aperture_2) / (math.pi*(r_3**2 - r_2**2)) 
print(b,'\n')

extra_cor = -b*math.pi*r_1**2
print(extra_cor)

# corrected sum
t_r = aperture_1-b*math.pi*r_1**2
# print(t_r)




In [ ]:
# determining the where the backgournd becomes constant as the radius increases
radius_arr = np.linspace(1,25,25)

ap_arr = []
b_arr = []
for i in radius_arr:
    ap = aperture_sum(i,counts_image[t,:,:], centroid_2dG_arr_x2[14], centroid_2dG_arr_y2[14])
    ap_arr.append(ap)
# print(ap_arr[0])

i = 0
for j in ap_arr:
#     print(j)
    if i == 0:
        v = (ap_arr[1] - ap_arr[0]) / (math.pi*(radius_arr[1]**2 - radius_arr[0]**2)) 
#         print(v)
        v = v +300
        i= 1+i
        b_arr.append(v)
    elif i > 0:
        v = (ap_arr[i-1] - ap_arr[i]) / (math.pi*(radius_arr[i-1]**2 - radius_arr[i]**2))
#         print(v)
        v = v +300
        i = 1+i
        b_arr.append(v)

# log_b_arr =float(b_arr)
# log_b_arr


plt.semilogy(radius_arr, b_arr)
# xlimit()
# plt.show()s
# print('background arrar as R increases:\n',b_arr)

# possible radius from 13 to 20

In [ ]:
r_1 = 7
r_2 = 13
r_3 = 23
t = 0
aperture_1 = aperture_sum(r_1,counts_image[t,:,:], centroid_2dG_arr_x2[14], centroid_2dG_arr_y2[14]) #total counts
aperture_2 = aperture_sum(r_2,counts_image[t,:,:], centroid_2dG_arr_x2[14], centroid_2dG_arr_y2[14])
aperture_3 = aperture_sum(r_3,counts_image[t,:,:], centroid_2dG_arr_x2[14], centroid_2dG_arr_y2[14])


#Area of aperture,
area_1 = math.pi * r_1**2
# Background in annulus from r_2 to r_3
background_counts = aperture_3 - aperture_2
# Area of Annulus
area_2 = math.pi*(r_3**2 - r_2**2) #also known as npix

# subtracting background: 
backgrnd_subtract = aperture_1 - background_counts*area_1/area_2
# print(backgrnd_subtract)


# print(background_counts/area_2)

print('radius 1:',aperture_1,'\nradius 2: ',aperture_2,'\nRadius 3:',aperture_3,'\n\n\n')
print('area one:',area_1,'\n')
print('background counts:\n',background_counts,'\n')
print('area two:',area_2,'\n')
print('background subtraction:\n',backgrnd_subtract)